In [1]:
!pip install keras-tuner -q

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

# Hyper Parameter Optimization
Keras Tuner allows you to replace hardcoded hyper-parameters with a *search space;* a range of possible hyperparameter choices.  

To specify a *search space*, you'll define a model building function that takes hyperparameter (hp) ranges of values.

In [3]:
def build_model(hp):
    units = hp.Int('units', min_value=16, max_value=64, step=32)
    model = keras.Sequential([
        layers.Dense(units, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    optimizer = hp.Choice('optimizer', values=['rmsprop','adam', 'sgd'])
    model.compile(optimizer=optimizer, loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


If you wish to adopt a more modular and configurable approach, you can also extend the HyperModle class and override the build method. This is useful if you want to use the same model architecture for different problems, or if you want to use the same model architecture with different hyperparameters.

In [4]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int('units', min_value=16, max_value=64, step=32)
        model = keras.Sequential([
            layers.Dense(units, activation='relu'),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        optimizer = hp.Choice('optimizer', values=['rmsprop','adam', 'sgd'])
        model.compile(optimizer=optimizer, loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

C:\Users\David\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


## Tuner
Next step is to define a *tuner* which is a for loop that will repeatedly...
> Pick a set of hyperparameter values
> Call the model-building function with these values to create a model
> Train the model and record its metrics

Several tuners are pre-built into kt such as *RandomSearch, BayesianOptimization, Hyperband.*

In [5]:
#build the tuner
tuner = kt.BayesianOptimization(build_model, objective='val_accuracy', max_trials=5, directory='mnist_kt_test', project_name='intro_to_kt_mnist', overwrite=True)

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam', 'sgd'], 'ordered': False}


In [7]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0
x_train_full = x_train
y_train_full = y_train


In [8]:
num_val_samples = 10000
x_train, x_val = x_train_full[:-num_val_samples], x_train_full[-num_val_samples:]
y_train, y_val = y_train_full[:-num_val_samples], y_train_full[-num_val_samples:]


In [9]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=callbacks)

Trial 5 Complete [00h 00m 37s]
val_accuracy: 0.9538000226020813

Best val_accuracy So Far: 0.9740999937057495
Total elapsed time: 00h 03m 40s
INFO:tensorflow:Oracle triggered exit


In [10]:
top_n = 3
top_params = tuner.get_best_hyperparameters(num_trials=top_n)
top_models = tuner.get_best_models(num_models=top_n)

In [14]:
def get_best_epoch(hp):
    model=build_model(hp)
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=10)]
    history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), batch_size=128, callbacks=callbacks)
    val_loss_per_epoch = history.history['val_loss']
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}" )
    return best_epoch


In [15]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    #model = build_model(hp)
    model.fit(x_train, y_train, epochs=int(best_epoch*1.2), validation_data=(x_val, y_val), batch_size=128)
    return model

In [16]:
best_models=[]
for hp in top_params:
    model=get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/50
391/391 [==============================] - 1s 2ms/step - loss: 0.4942 - accuracy: 0.8667 - val_loss: 0.2473 - val_accuracy: 0.9317
Epoch 2/50
391/391 [==============================] - 1s 2ms/step - loss: 0.2264 - accuracy: 0.9355 - val_loss: 0.1845 - val_accuracy: 0.9481
Epoch 3/50
391/391 [==============================] - 1s 2ms/step - loss: 0.1771 - accuracy: 0.9491 - val_loss: 0.1571 - val_accuracy: 0.9560
Epoch 4/50
391/391 [==============================] - 1s 2ms/step - loss: 0.1468 - accuracy: 0.9578 - val_loss: 0.1381 - val_accuracy: 0.9636
Epoch 5/50
391/391 [==============================] - 1s 1ms/step - loss: 0.1260 - accuracy: 0.9640 - val_loss: 0.1270 - val_accuracy: 0.9646
Epoch 6/50
391/391 [==============================] - 1s 2ms/step - loss: 0.1106 - accuracy: 0.9679 - val_loss: 0.1210 - val_accuracy: 0.9648
Epoch 7/50
391/391 [==============================] - 1s 1ms/step - loss: 0.0982 - accuracy: 0.9718 - val_loss: 0.1137 - val_accuracy: 0.9669
Epoch 

In [17]:
best_models = tuner.get_best_models(top_n)